In [ ]:
# DDP is different than DP
# DDP, each GPU (woker) has a copy of the model
# data is divided into batches
# each GPU gets its batch and do the forwrad pass
# 

In [ ]:
import os
import torch
import sys
import tempfile
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
import torch.multiprocessing as mp



from torch.nn.parallel import DistributedDataParallel 

In [ ]:
def setup(world_size, rank):

    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "75872"

    dist.init_process_group("nccl", world_size=world_size , rank= rank)

def cleanup():

    dist.destroy_process_group()



    


In [ ]:
class ToyModel(nn.Module):
    def __init__(self):
        super(ToyModel,self).__init__()
        self.net1 = nn.Linear(10, 10)
        self.relu1 = nn.ReLU()
        self.net2 = nn.Linear(10, 5)

    def forward(self, x):
        return self.net2(self.relu1(self.net1(x)))
    
    






In [ ]:
model = ToyModel()

In [ ]:
x = torch.randn(32, 10)

In [ ]:
out = model(x)

In [ ]:
out.shape

In [ ]:
optimizer = optim.Adamax(model.parameters(), lr = 1e-2)

In [ ]:
logits = model(x)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
loss = criterion(logits, labels)



In [ ]:
loss.backwards() # this is gradinets calcilations
optimizer.step() # this is weight upodate
optimizer.zero_grad() # this is clearinng optimzier states for iter


In [ ]:
import torch
import sys
import os
import tempfile
from torch.optim import adamw
import torch.distributed as dist
import torch.nn as nn

from torch.nn.parallel import DistributedDataParallel as DPP


In [ ]:
# A simple two linear feed forward network, it takes an input and passes 
# it through a layer with 10 hidden units 
# then it pass that through relu
# then it passes it through another hidden layer which gives 5 dimensional vecto
class ToyModel(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.net1 = nn.Linear(10,10)
        self.relu = nn.ReLU()
        self.net2 = nn.Linear(10, 5)
    
    def forward(self, x):
        return self.net2(self.relu(self.net1(x)))
    





In [ ]:
# set up DDP, so that each process know the address of a single machine for all reduce
# also we need to define world size and rank.

def ddp_setup(rank: int, world_size: int) -> None:
    os.environ["MASTER_ADDR"] = "localhost"  # or this should be the first node IP.
    os.environ["MASTER_PORT"] = "any port" # any available port


    dist.init_process_group("nccl", rank = rank, world_size= world_size)
    torch.cuda.set_device(rank) # this ensures that gpu0 gets rank 0 and so on.


def ddp_cleanup() -> None:
    """
    This is to delete the ddp setup after everything is done.
    """
    dist.destroy_process_group()

In [ ]:
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import TensorDataset, DistributedSampler, DataLoader
from torch.optim import AdamW
import torch.nn as nn
def run_epoch(rank:int, world_size: int) -> None:

    print(f"running rank {rank} and world size {world_size}")

    ddp_setup(rank, world_size) # getting 

    model = ToyModel().to(rank) # put the model on specific rank gpu
    model = DDP(model, device_ids = [rank]) # Wrap the model with DDP 
    # means that each paramters hook will be automatically initiated
    # and when the backward pass happens, for each process 
    # seperetely, when there is time of updatuing weights for each parameter
    # of each process, then the average ghradianet will be brought for that
    # parameter and it will be updated with that average gradient.
    # 

    # synthetic data random 1000 rows
    x = torch.randn(1000, 10)
    y = torch.randn(1000, 5)

    # encapsulate x and y in input output target pairs
    ds = TensorDataset(x, y)
    # Once we have the TensorDataset, we can use it with data loader ot distrbiuted data sampler
    # DistributedDataSampler makes sure that each batch is unique
    sampler = DistributedSampler(ds, num_replicas= world_size, rank = rank, 
                                 shuffle= True)
    
    # the way it works, x, y-->TensorDataset-->DistributedSampler-->DataLoader

    dl = DataLoader(ds, sampler = sampler, batch_size=64)

    ## the data is set now, 
    # ddp setup --> model --> model wrap to DDP--> data setup  (xy, ds, dl)
    optimizer = AdamW(model.parameters(), lr = 1e-4)
    loss_fn = nn.MSELoss()

    # Get exatly one epoch
    sampler.set_epoch(0)

    batch_x, batch_y = next(iter(dl))
    batch_x = batch_x.to(rank)
    batch_y = batch_y.to(rank)

    optimizer.zero_grad()
    predictions = model(batch_x)
    loss = loss_fn(input=predictions, target = batch_y)

    loss.backwards() # this is where hooks are fired and regisreed. gradoients 
    # are calculated 
    optimizer.step() # this is where the local gradients for each param in each rank us replaced
    # by the average all reduce operraion gradinet
    print(f" [rank {rank}] has a loss of {loss.item():.6f}", flush=True)
    ddp_cleanup()

    print(f"rank {rank} finished", flush=True)









    








In [ ]:
from datasets import  load_dataset

In [ ]:
raw = load_dataset("ag_news")

In [ ]:
raw

In [ ]:
raw["train"][3]

In [ ]:
import numpy as np
np.unique(raw["train"]['label'])

In [ ]:
# tokenization

from transformers import AutoTokenizer

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
cp = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(cp)

In [ ]:
def tokenization(example):
    return tokenizer(example["text"], truncation=True, max_length=128)

In [ ]:
ds_tokens = raw.map(tokenization, batched = True, remove_columns=["text"])

In [ ]:
ds_tokens

In [ ]:
train_ds = ds_tokens["train"]["input_ids"]

In [ ]:
len(train_ds)

In [ ]:
len(train_ds[0])

In [ ]:
column = "text" if "text" in raw["train"].column_names else list(raw["train"].column_names)[0]


In [ ]:
column

In [ ]:
remove_columns=[c for c in raw["train"].column_names if c != "label"]


In [ ]:
remove_columns

In [ ]:
ds_tokens.set_format(type= "torch")

In [ ]:
ds_tokens

In [ ]:
ds_tokens["train"][0]

In [ ]:
raw

In [ ]:
raw["train"].column_names[0]

In [ ]:
# if the first is text, then fine make that your input text otherwise the first one should be selected
column = "text" if "text" in raw["train"].column_names else raw["train"].column_names[0]



In [ ]:
column

In [ ]:
def tokenization(example):
    
    return tokenizer(example[column], truncation=True, max_length=128)


token_ds = raw.map(tokenization, batched=True, remove_columns=[column])

In [ ]:
token_ds.set_format(type="torch")

In [ ]:
token_ds['train'][0]

In [ ]:
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          DataCollatorWithPadding)

In [ ]:
collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors= "pt")

In [ ]:
raw["train"].features["label"].names

In [ ]:
lr_scheular.StepLT)

In [ ]:
import torch
import torch.distributed as dist
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          DataCollatorWithPadding)
from torch.nn.parallel import DistributedDataParallel
from datasets import load_dataset
import sys
import os
from torch.amp import autocast
import contextlib
from torch.utils.data import TensorDataset, DistributedSampler, DataLoader


In [ ]:
def ddp_setup(rank:int, world_size:int)->None:

    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)


def ddp_cleanup():
    dist.destroy_process_group()





In [ ]:


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
raw_data = load_dataset("ag_news")

column = "text" if "text" in raw_data["train"].column_names else raw_data["train"].column_names[0]



In [ ]:
column

In [ ]:
raw_data


In [ ]:
def tokenization(example):

    tokenizer(example[column], truncation= True, max_length=128)

In [ ]:
token_ds = raw_data.map(tokenization, batched=True, remove_columns=[column])

In [ ]:
token_ds

In [ ]:
token_ds["train"].features["label"].names

In [ ]:
token_ds["train"].features["label"]

In [ ]:
np.unique(token_ds["train"]["label"])

In [ ]:
# usual way 
# x = torch.randn(10, 10)
# y = torch.randn(10, 5)

# ds = TensorDataset(x , y)

# sampler = DistributedSampler( dataset=ds, num_replicas=world_size, rank = rank, shuffle=True)

# dl = DataLoader(dataset=ds, sampler=sampler, batch_size=32, collate_fn=collate)

In [ ]:
collate = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, return_tensors="pt")
# this will make sure we have padding across each batch and not the whole

In [ ]:
# we have the ds already
token_ds


In [ ]:

def tokenization(example, column):

    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

    return tokenizer(example[column])


def data_loader(rank:int, world_size:int, batch_size: int):

    raw = load_dataset("ag_news")
    column = "text" if "text" in raw["train"].column_names else raw["train"].column_names[0]

    token_ds = raw.map(tokenization, batched=True, remove_columns=[column])

    token_ds.set_format(type="torch")

    

    

    







sampler_train = DistributedSampler(dataset=token_ds["train"], num_replicas=world_size, 
                                   rank=rank, shuffle=True, drop_last=True)

sampler_test = DistributedSampler(dataset=token_ds["test"], num_replicas=world_size, 
                                   rank=rank, shuffle=False, drop_last=False)